In [1]:
# Get Data to pass through SMOTE

import pandas as pd

df = pd.read_csv("./data/curData.csv")
df = df.drop(columns=["issue_d"], axis=1)
df['issue_month'] = pd.to_datetime(df['issue_month'])
df["issue_month"] = df["issue_month"].dt.month
df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], axis=1)
df["purpose"] = df["purpose"].astype("category")
df["home_ownership"] = df["home_ownership"].astype("category")
feature_cols = ['loan_amnt', 'term', 'int_rate',
       'installment', 'purpose', 'emp_length', 'home_ownership', 'annual_inc',
       'dti', 'inq_last_6mths', 'delinq_2yrs', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'issue_month',
       'installment_to_income', 'fico_score', 'credit_age',
       'fed_funds_rate', 'unemployment_rate', 'cpi', 'real_gdp',
       'debt_service_ratio']

target = "loan_risk"

In [ ]:
for col in df.columns:
    if df[col].dtype == "category":
        print(col)

purpose
home_ownership


In [18]:
purpose_cat = df["purpose"].cat.categories
df["purpose_codes"] = df["purpose"].cat.codes
df = df.drop(columns=["purpose"])

home_ownership_cat = df["home_ownership"].cat.categories
df["home_ownership_codes"] = df["home_ownership"].cat.codes
df = df.drop(columns=["home_ownership"])

C:\Users\Jayden Spurgiasz\AppData\Local\Temp\ipykernel_27464\2732330225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["purpose_codes"] = df["purpose"].cat.codes


In [19]:
df = df.dropna()

In [21]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')

smote_cols = ['loan_amnt', 'term', 'int_rate',
       'installment', 'purpose_codes', 'emp_length', 'home_ownership_codes', 'annual_inc',
       'dti', 'inq_last_6mths', 'delinq_2yrs', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'issue_month',
       'installment_to_income', 'fico_score', 'credit_age',
       'fed_funds_rate', 'unemployment_rate', 'cpi', 'real_gdp',
       'debt_service_ratio']

X = df[smote_cols]
y = df[target]

X, y = smote.fit_resample(X, y)
y.value_counts()

loan_risk
0    1966639
1    1966639
Name: count, dtype: int64

In [22]:
smote_df = X
smote_df[target] = y

In [23]:
smote_df["purpose"] = pd.Categorical.from_codes(smote_df["purpose_codes"], categories=purpose_cat)
smote_df["home_ownership"] = pd.Categorical.from_codes(smote_df["home_ownership_codes"], categories=home_ownership_cat)

In [25]:
smote_df["purpose"]

0          debt_consolidation
1              small_business
2            home_improvement
3          debt_consolidation
4              major_purchase
                  ...        
3933273                 other
3933274    debt_consolidation
3933275    debt_consolidation
3933276    debt_consolidation
3933277    debt_consolidation
Name: purpose, Length: 3933278, dtype: category
Categories (14, object): ['car', 'credit_card', 'debt_consolidation', 'educational', ..., 'renewable_energy', 'small_business', 'vacation', 'wedding']

In [26]:
smote_df = smote_df.drop(columns=["purpose_codes", "home_ownership_codes"])

In [27]:
smote_df.to_csv("./data/smoteData.csv")